In [153]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [154]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [155]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
import numpy as np

vectorizer = CountVectorizer()

finallist=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file(filename,createfile):
    tokenize_words=[]
    maintext=[]
    listofimpcandidate=[]
    listwithfinalsentence=[]
    file1 = open(createfile, "w", encoding="utf-8")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    titleno = soup.find_all('title')
    text=soup.find_all('text')
    instances = {}
    my_id = 0
    for n,tit,maintitle in zip(docno,text,titleno):
        tit=tit.get_text()
        tit=str(tit)
        sentences=sentence_tokenize.sentence_split(tit, lang='gu')
        for t in sentences:
            sen=removestopwords(t)
            #print("this is sen",sen)
            maintext.append(sen)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in maintext]
        list3 = [s for s in list3 if s]
        
        bag_of_words = vectorizer.fit_transform(list3)
        bag_of_words.todense()
        #This process encodes our original data into topic encoded data
        svd = TruncatedSVD(n_components = 2)
        lsa = svd.fit_transform(bag_of_words)
        topic_encoded_df = pd.DataFrame(lsa, columns=["topic1", "topic2"])
        topic_encoded_df["doc"]= list3
        avg1=topic_encoded_df["topic1"].mean()
        avg2=topic_encoded_df["topic2"].mean()
        topic_encoded_df['topic1'] = np.where(topic_encoded_df['topic1'] < avg1, 0, topic_encoded_df['topic1'])
        topic_encoded_df['topic2'] = np.where(topic_encoded_df['topic2'] < avg2, 0, topic_encoded_df['topic2'])
        sum_column = topic_encoded_df["topic1"] + topic_encoded_df["topic2"]
        topic_encoded_df["sum_column"]=sum_column
        sort_by_sumcolumn = topic_encoded_df.sort_values('sum_column',ascending=False)
        compressionratio = round((len(sort_by_sumcolumn.index)*50)/100)
        #print(compressionratio)
        final_df2=sort_by_sumcolumn['doc'].head(compressionratio)
        listtoretrieve=list(final_df2.index.values)
        listtoretrieve.sort()
        originalsen=[]
        for t in sentences:    
            originalsen.append(t)
        df4 = pd.DataFrame(originalsen,columns=['sentences'])
        #print(df4)
        finaldraft=""
        for i in listtoretrieve:
            finaldraft+=df4['sentences'].loc[i]
        #print(finaldraft)            
#         originalfull_str = convert_list_to_string(finaldraft)
#         originalfull_str=" ".join(originalfull_str.split())
#         n=str(n)
        file1.write(finaldraft)
        instances = {}    
        
    return instances

In [156]:
import os

#for root, dirs, files in os.walk("/home/harsh/Documents/gujdocs/gsf2003/"):
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization dataset/entertainment/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file(os.path.join(root, file),"G:/Movies/gujarati Text summarization_lda/entertainment/"+file)

In [157]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level

In [158]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [159]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
rouge1=[]
rouge2=[]
rougel=[]
rougew=[]

class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file1(filename,reference):
    tokenize_words=[]
    tokenize_words1=[]
    
    #file1 = open(reference, "r")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    #print(data)
    
    for tokenize in indic_tokenize.trivial_tokenize(data):
            tokenize_words.append(tokenize)
        # Convert list of strings to string
    originalfull_str = convert_list_to_string(tokenize_words)
    senlateron=[]
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    i=1
    with codecs.open(reference+"a2_"+str(i)+".txt", encoding='utf-8') as f1:
        data1 = f1.read()
    i=i+1
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    senlateron1=[]
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-W: %f' % rouge_w)
    rougew.append(rouge_w)
    print(rouge1)
    average1 = Average(rouge1)
    print("average of rouge1",average1)
    print(rouge2)
    average2 = Average(rouge2)
    print("average of rouge2",average2)
    print(rougel)
    averagel = Average(rougel)
    print("average of rougel",averagel)
    print(rougew)
    averagew = Average(rougew)
    print("average of rougew",averagew)
   
    instances = {}


    return instances

In [160]:
import os
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization_lda/entertainment/"):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file1(os.path.join(root, file),"C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/entertainment/annotator2/")

[0.8419301164725458]
average of rouge1 0.8419301164725458
[0.7679465776293823]
average of rouge2 0.7679465776293823
[0.8419301164725458]
average of rougel 0.8419301164725458
[0.3672924037013795]
average of rougew 0.3672924037013795
[0.8419301164725458, 0.5988165680473373]
average of rouge1 0.7203733422599415
[0.7679465776293823, 0.3013048635824436]
average of rouge2 0.534625720605913
[0.8419301164725458, 0.5940828402366863]
average of rougel 0.7180064783546161
[0.3672924037013795, 0.2618086743525092]
average of rougew 0.31455053902694435
[0.8419301164725458, 0.5988165680473373, 0.46125461254612543]
average of rouge1 0.6340004323553362
[0.7679465776293823, 0.3013048635824436, 0.23659889094269868]
average of rouge2 0.4352834440515082
[0.8419301164725458, 0.5940828402366863, 0.45756457564575653]
average of rougel 0.6311925107849963
[0.3672924037013795, 0.2618086743525092, 0.21346704517063703]
average of rougew 0.2808560410748419
[0.8419301164725458, 0.5988165680473373, 0.46125461254612543

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866]
average of rouge1 0.6037623842713513
[0.7679465776293823, 0.3013048635824436, 0.23659889094269868, 0.2609457092819614, 0.2897102897102898, 0.2777155655095185, 0.2906474820143885, 0.28994708994708995, 0.32083958020989506, 0.33234421364985167, 0.2843450479233227, 0.28152492668621704, 0.28571428571428575, 0.2836004932182491]
average of rouge2 0.32165607257282813
[0.8419301164725458, 0.5940828402366863, 0.45756457564575653, 0.458041958041958, 0.5104685942173479, 0.576536312849162, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.6910828025477708, 0.5092682926829268, 0.6338215712383488, 0.5904059040590406]
average of rougel 0.601497263942501
[0.3672924037013795, 0.2618086743525092, 0.2

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184]
average of rouge1 0.5808510661046968
[0.7679465776293823, 0.3013048635824436, 0.23659889094269868, 0.2609457092819614, 0.2897102897102898, 0.2777155655095185, 0.2906474820143885, 0.28994708994708995, 0.32083958020989506, 0.33234421364985167, 0.2843450479233227, 0.28152492668621704, 0.28571428571428575, 0.2836004932182491, 0.23266932270916335, 0.31442786069651746, 0.2534948741845294, 0.32095096582466565, 0.26078234704112335, 0.30750893921334926]
average of rouge2 0.3096509662844471
[0.8419301164725458, 0.5940828402366863, 0.45756457564575653, 0.458041958041958, 0.5104685942173479, 0.5765363128491

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633]
average of rouge1 0.5945600752514724
[0.7679465776293823, 0.3013048635824436, 0.23659889094269868, 0.2609457092819614, 0.2897102897102898, 0.2777155655095185, 0.2906474820143885, 0.28994708994708995, 0.32083958020989506, 0.33234421364985167, 0.2843450479233227, 0.28152492668621704, 0.28571428571428575, 0.2836004932182491, 0.23266932270916335, 0.31442786069651746, 0.2534948741845294, 0.32095096582466565, 0.26078234704112335, 0.30750893921334926, 0.2967741935483871, 0.28528072837632773, 0.3297709923

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633, 0.0, 0.6560000000000001, 0.6372688477951636, 0.44502164502164504]
average of rouge1 0.5724928404863316
[0.7679465776293823, 0.3013048635824436, 0.23659889094269868, 0.2609457092819614, 0.2897102897102898, 0.2777155655095185, 0.2906474820143885, 0.28994708994708995, 0.32083958020989506, 0.33234421364985167, 0.2843450479233227, 0.28152492668621704, 0.28571428571428575, 0.2836004932182491, 0.23266932270916335, 0.31442786069651746, 0.2534948741845294, 0.32095096582466565, 0.26078234704112335, 0.307508

...

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633, 0.0, 0.6560000000000001, 0.6372688477951636, 0.44502164502164504, 0.38786084381939306, 0.7027027027027026, 0.5701559020044543, 0.45895851721094444, 0.7123287671232875, 0.7069767441860466, 0.7411167512690355, 0.49085659287776706, 0.4619666048237477, 0.5191313340227508, 0.6225680933852141, 0.7160883280757098, 0.6966966966966966, 0.7252747252747253, 0.6546961325966851, 0.524390243902439, 0.508, 0.5537098560354374, 0.5921521997621878, 0.7044776119402985, 0.31899641577060933, 0.5458422174840086, 0.4414

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633, 0.0, 0.6560000000000001, 0.6372688477951636, 0.44502164502164504, 0.38786084381939306, 0.7027027027027026, 0.5701559020044543, 0.45895851721094444, 0.7123287671232875, 0.7069767441860466, 0.7411167512690355, 0.49085659287776706, 0.4619666048237477, 0.5191313340227508, 0.6225680933852141, 0.7160883280757098, 0.6966966966966966, 0.7252747252747253, 0.6546961325966851, 0.524390243902439, 0.508, 0.5537098560354374, 0.5921521997621878, 0.7044776119402985, 0.31899641577060933, 0.5458422174840086, 0.4414

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633, 0.0, 0.6560000000000001, 0.6372688477951636, 0.44502164502164504, 0.38786084381939306, 0.7027027027027026, 0.5701559020044543, 0.45895851721094444, 0.7123287671232875, 0.7069767441860466, 0.7411167512690355, 0.49085659287776706, 0.4619666048237477, 0.5191313340227508, 0.6225680933852141, 0.7160883280757098, 0.6966966966966966, 0.7252747252747253, 0.6546961325966851, 0.524390243902439, 0.508, 0.5537098560354374, 0.5921521997621878, 0.7044776119402985, 0.31899641577060933, 0.5458422174840086, 0.4414

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633, 0.0, 0.6560000000000001, 0.6372688477951636, 0.44502164502164504, 0.38786084381939306, 0.7027027027027026, 0.5701559020044543, 0.45895851721094444, 0.7123287671232875, 0.7069767441860466, 0.7411167512690355, 0.49085659287776706, 0.4619666048237477, 0.5191313340227508, 0.6225680933852141, 0.7160883280757098, 0.6966966966966966, 0.7252747252747253, 0.6546961325966851, 0.524390243902439, 0.508, 0.5537098560354374, 0.5921521997621878, 0.7044776119402985, 0.31899641577060933, 0.5458422174840086, 0.4414

[0.8419301164725458, 0.5988165680473373, 0.46125461254612543, 0.458041958041958, 0.5124626121635095, 0.5810055865921787, 0.6743185078909613, 0.515311510031679, 0.6935724962630793, 0.6745562130177515, 0.694267515923567, 0.5151219512195122, 0.6391478029294275, 0.5928659286592866, 0.4089101034208433, 0.522343594836147, 0.4762790697674419, 0.6755555555555556, 0.5105105105105104, 0.5707491082045184, 0.697749196141479, 0.7291981845688351, 0.7153729071537291, 0.47174887892376677, 0.6329113924050633, 0.0, 0.6560000000000001, 0.6372688477951636, 0.44502164502164504, 0.38786084381939306, 0.7027027027027026, 0.5701559020044543, 0.45895851721094444, 0.7123287671232875, 0.7069767441860466, 0.7411167512690355, 0.49085659287776706, 0.4619666048237477, 0.5191313340227508, 0.6225680933852141, 0.7160883280757098, 0.6966966966966966, 0.7252747252747253, 0.6546961325966851, 0.524390243902439, 0.508, 0.5537098560354374, 0.5921521997621878, 0.7044776119402985, 0.31899641577060933, 0.5458422174840086, 0.4414